In [ ]:
import numpy as np
import tkinter.filedialog as tkf
import tifffile
import hyperspy.api as hys

In [ ]:
ref_adr = tkf.askopenfilename()
print(ref_adr)

In [ ]:
si_adr = tkf.askopenfilename()
print(si_adr)

In [ ]:
ref_zlp = hys.load(ref_adr)[1]
print(ref_zlp)

In [ ]:
si = hys.load(si_adr)
print(si)

In [ ]:
step = ref_zlp.axes_manager[0].scale
left = ref_zlp.axes_manager[0].offset
e_size = ref_zlp.axes_manager[0].size
print(left, step, e_size)
x_range = np.arange(left, e_size*step+left, step)
print(x_range[0], x_range[-1])
print(x_range.shape)

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
roi_range = [-1.0, 1.0]
roi_index = [find_nearest(x_range, roi_range[0]), find_nearest(x_range, roi_range[1])]
print(roi_index)

In [ ]:
zlp_data = ref_zlp.data
print(zlp_data.shape)
ref_marg = np.argmax(zlp_data)
ref_mint = np.max(zlp_data)
print(ref_marg, ref_mint)

In [ ]:
si_data = si.data
si_shape = si_data.shape
print(si_shape)

In [ ]:
diff_si = []
for i in range(si_shape[0]):
    for j in range(si_shape[1]):
        tmp = si_data[i, j].copy()
        tmp_max = np.max(tmp)
        tmp -= zlp_data[roi_index[0]:roi_index[1]]
        diff_si.append(tmp)

In [ ]:
diff_si = np.asarray(diff_si).reshape(si_shape[0], si_shape[1], -1)
print(diff_si.shape)

In [ ]:
tifffile.imsave(tkf.asksaveasfilename(), diff_si)

In [ ]:
half_max = ref_mint / 2
print(half_max)
fwhm_lind = find_nearest(zlp_data[:ref_marg], half_max)
fwhm_rind = find_nearest(zlp_data[ref_marg:], half_max) + ref_marg
print(fwhm_lind, fwhm_rind)
fwhm_integral = np.sum(zlp_data[fwhm_lind:fwhm_rind+1])
print(fwhm_integral)

In [ ]:
diff_si_fwhm = []
for i in range(si_shape[0]):
    for j in range(si_shape[1]):
        tmp = si_data[i, j].copy()
        tmp_marg = np.argmax(tmp)
        tmp_mint = np.max(tmp)
        #print(tmp_marg, tmp_mint)
        
        tmp_half_max = tmp_mint / 2
        #print(tmp_half_max)
        tmp_lind = find_nearest(tmp[:tmp_marg], tmp_half_max)
        tmp_rind = find_nearest(tmp[tmp_marg:], tmp_half_max) + tmp_marg
        #print(tmp_lind, tmp_rind)
        tmp_integral = np.sum(tmp[tmp_lind:tmp_rind+1])
        #print(tmp_integral)
        
        correction_term = tmp_integral / fwhm_integral
        tmp -= zlp_data * correction_term
        
        diff_si_fwhm.append(tmp)        

In [ ]:
diff_si_fwhm = np.asarray(diff_si_fwhm).reshape(si_shape[0], si_shape[1], -1)
print(diff_si_fwhm.shape)

In [ ]:
tifffile.imsave(tkf.asksaveasfilename(), diff_si_fwhm)